<a href="https://colab.research.google.com/github/LeeSeungYun1020/HandsOnLargeLanguageModels/blob/main/handsOnLLM04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 분류

- 핸즈온 LLM 4장
- 모델을 훈련하여 입력 텍스트에 레이블 또는 클래스를 할당
- 감성 분석, 의도 감지, 엔티티 추출, 언어 감지에 사용
- 생각보다 간단하지 않으며, 창의적인 기법이 매우 많음

## 영화 리뷰 데이터셋

- 테스트해 볼 텍스트는 로튼 토마토 데이터셋!
	- 이진 감성 분류 작업
	- 긍정, 부정 영화 리뷰 데이터 각 5331개
	- 훈련 데이터와 검증 데이터로 구성

In [1]:
from datasets import load_dataset

data = load_dataset("rotten_tomatoes")
data

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [3]:
data['train'][0, -1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

## 표현 모델로 텍스트 분류

- 표현 모델은 텍스트를 잘 나타내는 카테고리를 바로 반환
- 생성 모델은 프롬프트를 통해 분류하도록 지시하면 출력으로 설명을 생성

- BERT 같은 파운데이션 모델을 훈련하여 미세 튜닝 -> 11장
- 임베딩 모델은 범용 목적의 임베딩 생성하므로 분류에 국한되지 않고 다양한 작업에 사용될 수 있음 -> 10장
- 여기에서는 두 모델을 직접 훈련하지 않고 사전 훈련된 모델을 사용하여 분류 진행

## 모델 선택

- 언어 호환성, 내부 구조, 크기, 성능을 고려하여 선택
- 인코더 기반 모델은 특정 작업에서 작은 크기로 뛰어난 성능을 냄(효율적, 생성 모델이 더 뛰어난 성능을 낼 수도 있음)
- BERT, RoBERTa, DistilBERT, DeBERTa, bert-tiny, ALBERT V2 등

## 작업에 특화된 모델 사용하기

1. Twitter-RoBERTa-base 모델
2. distilbert-base-uncased-finetuned-sst-2-english 모델

In [4]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative = output[0]["score"]
    positive = output[2]["score"]
    assignment = np.argmax([negative, positive])
    y_pred.append(assignment)


100%|██████████| 1066/1066 [00:25<00:00, 41.40it/s]


In [6]:
from sklearn.metrics import classification_report


def evaluate(y_true, y_pred):
    print(classification_report(y_true, y_pred, target_names=["Negative Review", "Positive Review"]))

In [8]:
evaluate(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



- 혼동 행렬
    - 정밀도: TP / (TP + FP), 양성으로 예측된 샘플 중 진양성 샘플 비율
    - 재현율: TP / (TP + FN), 전체 양성 샘플 중 진양성 샘플 비율
    - 정확도: TP + TN / (TP + TN + FP + FN), 전체 샘플 중 올바르게 예측된 샘플 비율
    - F1 점수: 정밀도, 재현율을 이용한 성능 평가 척도

In [13]:
model_path = "distilbert-base-uncased-finetuned-sst-2-english"

pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
  negative = output[0]["score"]
  positive = output[1]["score"]
  assignment = np.argmax([negative, positive])
  y_pred.append(assignment)

evaluate(data["test"]["label"], y_pred)

Device set to use cuda:0
100%|██████████| 1066/1066 [00:05<00:00, 186.95it/s]


                 precision    recall  f1-score   support

Negative Review       0.89      0.90      0.90       533
Positive Review       0.90      0.89      0.90       533

       accuracy                           0.90      1066
      macro avg       0.90      0.90      0.90      1066
   weighted avg       0.90      0.90      0.90      1066



- 결과
  1. Twitter-RoBERTa-base 모델: 0.80
  2. distilbert-base-uncased-finetuned-sst-2-english 모델: 0.90
  
  


.    

## 임베딩을 활용하여 분류 작업 수행

- 특정 작업에 맞는 사전 훈련된 모델이 없으면 어떻게 해야 하나?
  - 직접 미세 튜닝 -> 가능하지만 많은 리소스 필요
  - 범용 임베딩 모델 사용

### 지도 학습 분류

- 지도 학습: 정답이 있는 데이터(라벨링된 데이터)로 학습
- 임베딩 모델로 특성을 생성 후 분류기(Ex: 회귀 모델) 사용

In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

type(data["train"]["text"])
train_embeddings = model.encode(data["train"]["text"][:], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"][:], show_progress_bar=True)

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [23]:
train_embeddings.shape

(8530, 768)

In [24]:
test_embeddings.shape

(1066, 768)

In [25]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

LogisticRegression(random_state=42)

In [26]:
y_pred = clf.predict(test_embeddings)
evaluate(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



- 결과
    1. SentenceTransformer + LogisticRegression: 0.85

### 데이터에 레이블이 없는 경우

감성을 분류하기 위해 입력 텍스트에 레이블(클래스)을 할당

- 제로샷 분류: 레이블이 없는 데이터로 작업 가능성을 가늠해 봄
  - 레이블 정의는 알지만 레이블이 없는 경우
  - 해당 데이터로 훈련되지 않았더라도 입력 데이터 레이블 예측 가능
  - 여기서는 긍정적 리뷰, 부정적 리뷰로 타깃 레이블 생성

In [51]:
label_embeddings = model.encode(["negative review", "positive review"])
# negative, positive 순서가 바뀌면 매우 안 좋은 결과 반환 / 0.22

- 코사인 유사도: 두 벡터 사이 각도의 코사인 값을 이용하여 유사도 판단
- 코사인 유사도를 계산하고 가장 높은 값을 골라 유사도 분석

In [52]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(similarity_matrix, axis=1)

In [53]:
evaluate(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.77      0.77      0.77       533
Positive Review       0.77      0.77      0.77       533

       accuracy                           0.77      1066
      macro avg       0.77      0.77      0.77      1066
   weighted avg       0.77      0.77      0.77      1066



In [56]:
# 강조해 보기
label_embeddings = model.encode(["A very negative movie review", "A very positive movie review"])
similarity_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(similarity_matrix, axis=1)
evaluate(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.86      0.73      0.79       533
Positive Review       0.76      0.88      0.82       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



- 결과
  1. cosine_similarity: 0.80


## 생성 모델로 텍스트 분류

- 작업에 특화된 모델은 시퀀스 투 밸류 모델로 값을 반환
- 생성 모델은 시퀀스 투 시퀀스 모델로 다른 시퀀스를 생성

- 아무 맥락 없이 영화 리뷰를 주면 문제를 수행하지 못하므로 맥락을 이해할 수 있도록 프롬프트를 제공해야 함
- 프롬프트 엔지니어링: 원하는 출력을 얻도록 프롬프트를 반복적으로 개선하는 과정

### T5 모델 사용

- 12개 인코더, 12개 디코더로 구성
- 사전 훈련: 마스크드 언어 모델링
- 미세 튜닝: 각 작업을 시퀀스-투-시퀀스 작업으로 변환 + 훈련
- GPT 모델과 비슷한 수준으로 명령 따름

In [41]:
pipe = pipeline(
	"text2text-generation",
	model="google/flan-t5-base",
	device="cuda:0",
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [42]:
# 생성 모델이므로 프롬프트 추가해야 함
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example["text"]})
data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [45]:
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
	text = output[0]["generated_text"]
	y_pred.append(0 if "negative" in text else 1)

100%|██████████| 1066/1066 [01:46<00:00,  9.98it/s]


In [46]:
evaluate(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.87      0.92      0.89       533
Positive Review       0.91      0.86      0.88       533

       accuracy                           0.89      1066
      macro avg       0.89      0.89      0.89      1066
   weighted avg       0.89      0.89      0.89      1066



- 결과
  1. T5 모델: 0.89

### ChatGPT 사용

- Closed 소스 모델로 구조는 알려지지 않았지만 이름에서 디코더 기반으로 추측
- 사전 훈련: 지시 튜닝 - 입력 프롬프트와 기대 출력 제공하여 프롬프트 바탕으로 출력 생성
- 미세 튜닝: 선호도 튜닝 - 만들어진 모델로 출력을 생성하여 수동으로 순위를 매기고 이를 사용하여 최종 모델 생성

- 유료 API라 여기서는 테스트하지 않았지만 저자의 테스트에 따르면 GPT-3.5 모델 F1 점수는 0.92
- 훈련된 데이터를 알지 못하므로 정확한 평가는 불가(모델이 이 데이터셋에서 훈련되었을 수 있음)